In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from mmf_speckle import *
from __init__ import *

In [3]:
import torch
import matplotlib.pyplot as plt
import h5py
import time

In [7]:
device="cuda"
dtype=torch.float32
isfloat64=False
root="./results/exp2"

### Exp2: add nosie

In [5]:
A = get_mmf_speckle_measure_matrix(0,device,dtype)
PM_image = get_pre_measure_img(device, dtype).reshape((-1,1)) 

In [11]:
# add nosie
nosie=True
nosie_sigma_list = [0,1,5,10]

In [12]:
for nosie_sigma in nosie_sigma_list:
    # unknow measurement
    A_u = get_mmf_speckle_measure_matrix(25,device,dtype)
    # reconstruction of two unknown images
    names=[
        "GI",
        "Baboon"
    ]
    y_u_list = []
    x_u_list = []
    for name in names:
        x_u = get_gi_image(0,device, dtype) if name=="GI" else get_t_image(name,device, dtype)
        y_u = speckle_measure(A_u, x_u.reshape((-1,1)), nosie=nosie, nosie_sigma=nosie_sigma)
        x_u_list.append(x_u)
        y_u_list.append(y_u)
        
    # Iterative Algorithm 2    
    for j in range(len(names)):
        name = names[j]
        f = h5py.File("{}/algo2/{}/{}.h5".format(root,nosie_sigma,name),"a")

        recv_u_idx = j
        x_u = x_u_list[recv_u_idx].reshape((-1,1))

        params_2 = {
            "A": A,
            "PM_image": PM_image,
            "PM_epoch": 20,
            "epoch": 200,
            "speckle_measure": lambda A: speckle_measure(A, x_u, nosie=nosie, nosie_sigma=nosie_sigma),
            "A_recv":None,
            "e_y":None,
            "isfloat64":isfloat64,
            "calc_error": lambda A: A @ x_u
        }

        decay_k = calc_iterative_algo_k(params_2["A"], params_2["PM_image"], x_u) 
        y_u = y_u_list[recv_u_idx]
        (A_recv, k), pm_errors, errors, times = iterative_algo_2(y_u, params_2, verbose=False)
        
        print(errors[-1])
        print("done")
        
        f.create_dataset("A_recv", data=A_recv.cpu().numpy())
        f.create_dataset("y_u", data=y_u.cpu().numpy())

        f.create_dataset("errors", data=np.array(errors))
        f.create_dataset("pm_errors", data=np.array(pm_errors))

        f.create_dataset("decay_k", data=decay_k.cpu().numpy())
        f.create_dataset("times", data=np.array([times]))   

        f.create_dataset("PM_epoch", data=np.array([params_2["PM_epoch"]]))  
        f.create_dataset("epoch", data=np.array([params_2["epoch"]]))   

        f.close()

tensor([[0.2630],
        [0.2627],
        [0.2637],
        ...,
        [0.2635],
        [0.2630],
        [0.2625]], device='cuda:0')
0.10062754
done
tensor([[1.0154],
        [1.0160],
        [1.0149],
        ...,
        [1.0150],
        [1.0149],
        [1.0153]], device='cuda:0')
0.11205996
done
tensor([[0.2630],
        [0.2628],
        [0.2639],
        ...,
        [0.2636],
        [0.2633],
        [0.2626]], device='cuda:0')
0.81993693
done
tensor([[1.0154],
        [1.0162],
        [1.0153],
        ...,
        [1.0154],
        [1.0148],
        [1.0155]], device='cuda:0')
0.8167418
done
tensor([[0.2625],
        [0.2631],
        [0.2652],
        ...,
        [0.2647],
        [0.2628],
        [0.2620]], device='cuda:0')
4.149123
done
tensor([[1.0165],
        [1.0160],
        [1.0178],
        ...,
        [1.0148],
        [1.0159],
        [1.0150]], device='cuda:0')
3.9544618
done
tensor([[0.2659],
        [0.2652],
        [0.2632],
        ...,
       

In [14]:
for nosie_sigma in nosie_sigma_list:
    # unknow measurement
    A_u = get_mmf_speckle_measure_matrix(25,device,dtype)
    # reconstruction of two unknown images
    names=[
        "GI",
        "Baboon"
    ]
    y_u_list = []
    x_u_list = []
    for name in names:
        x_u = get_gi_image(0,device, dtype) if name=="GI" else get_t_image(name,device, dtype)
        y_u = speckle_measure(A_u, x_u.reshape((-1,1)), nosie=nosie, nosie_sigma=nosie_sigma)
        x_u_list.append(x_u)
        y_u_list.append(y_u)
        
    # Iterative Algorithm 2    
    for j in range(len(names)):
        name = names[j]
        f = h5py.File("{}/algo1/{}/{}.h5".format(root,nosie_sigma,name),"a")

        recv_u_idx = j
        x_u = x_u_list[recv_u_idx].reshape((-1,1))

        params_1 = {
            "A": A,
            "PM_image": PM_image,
            "epoch": 200,
            "speckle_measure": lambda A: speckle_measure(A, x_u, nosie=nosie, nosie_sigma=nosie_sigma),
            "A_recv":None,
            "e_y":None,
            "calc_error": lambda A: A @ x_u
        }
        decay_k = calc_iterative_algo_k(params_1["A"], params_1["PM_image"], x_u) 
        y_u = y_u_list[recv_u_idx]
        A_recv, errors = iterative_algo_1(y_u, params_1, verbose=False)
        
        print(errors[-1])
        print("done")
        
        f.create_dataset("A_recv", data=A_recv.cpu().numpy())
        f.create_dataset("y_u", data=y_u.cpu().numpy())
        f.create_dataset("errors", data=np.array(errors))
        f.create_dataset("decay_k", data=decay_k.cpu().numpy())
        f.close()

5.4541015e-05
done
8.59375e-05
done
0.31311536
done
0.8405421
done
1.5698551
done
4.053559
done
3.1980915
done
8.570801
done
